In [21]:
import pandas as pd
import numpy as np

In [22]:
data_folder = "C:\\Users\\spars\\Documents\\Master\\JHU\TML\\HomePriceBeastNew\\"

In [23]:
merged_home_data_time_series = pd.read_csv(f"{data_folder}merged_home_data_time_series_sale_model.csv", low_memory=False, thousands=',')

In [24]:
def transform_types_and_chunk(data, non_float_cols, date_col, dummy_cols):
    for x in data.columns:
        if x not in non_float_cols:
            data[x] = pd.to_numeric(data[x])
            
    data = pd.get_dummies(data, columns = dummy_cols)

    data[date_col] = pd.to_datetime(data[date_col])
    post_covid_frame = data[data[date_col] >= "2020-03-01"]
    train_frame = data[data[date_col] < "2019-12-01"]
    test_frame = data[((data[date_col] >= "2019-12-01") & (data[date_col] < "2020-03-01"))]
    
    return train_frame, test_frame, post_covid_frame

In [25]:
non_float_cols = ["state_code", "county_name", "period_begin", "state_code_dummy"]

#convert these columns to encoding. 
dummy_cols = ["state_code_dummy"]

#Keep original state code data after dummify
merged_home_data_time_series['state_code_dummy'] = merged_home_data_time_series['state_code']

train_frame, test_frame, post_covid_frame = transform_types_and_chunk(merged_home_data_time_series,
                                 non_float_cols,
                                 "period_begin",
                                 dummy_cols)

In [26]:
post_covid_frame.to_csv(f"{data_folder}post_covid_frame_df_sale_model.csv")
test_frame.to_csv(f"{data_folder}test_frame_df_sale_model.csv")

In [27]:
pd.set_option("display.max_columns", None)

In [28]:
test_frame.head(3)

,state_code,county_name,period_begin,median_sale_price,median_sale_price_lag_1,median_sale_price_lag_2,median_sale_price_lag_3,median_sale_price_lag_4,median_sale_price_lag_5,median_sale_price_lead_1,median_sale_price_lead_2,median_sale_price_lead_3,inventory,inventory_lag_1,inventory_lag_2,inventory_lag_3,inventory_lag_4,inventory_lag_5,inventory_lead_1,inventory_lead_2,inventory_lead_3,R_INTERNATIONAL_MIG_2019,Unemployment_rate_2020,PCT_COLL_4_2015_19,PCT_COLL_1TO3_2000,PCT_HSD_Only_2000,R_NET_MIG_2019,Med_HH_Income_Percent_of_State_Total_2019,GQ_ESTIMATES_2019,N_POP_CHG_2019,INTERNATIONAL_MIG_2019,NET_MIG_2019,HSD_Only_2000,DOMESTIC_MIG_2019,RESIDUAL_2019,Deaths_2019,COLL_4_2000,POP_ESTIMATE_2019,LT_HSD_2015_19,COLL_1TO3_2000,Unemployed_2020,NATURAL_INC_2019,GQ_ESTIMATES_BASE_2010,Employed_2020,LT_HSD_2000,COLL_4_2015_19,HSD_Only_2015_19,COLL_1TO3_2015_19,Civilian_labor_force_2020,CENSUS_2010_POP,PCT_LT_HSD_2000,R_birth_2019,PCT_COLL_1TO3_2015_19,PCT_COLL_4_2000,Economic_typology_2015,R_death_2019,state_code_dummy_AK,state_code_dummy_AL,state_code_dummy_AR,state_code_dummy_AZ,state_code_dummy_CA,state_code_dummy_CO,state_code_dummy_CT,state_code_dummy_DC,state_code_dummy_DE,state_code_dummy_FL,state_code_dummy_GA,state_code_dummy_HI,state_code_dummy_IA,state_code_dummy_ID,state_code_dummy_IL,state_code_dummy_IN,state_code_dummy_KS,state_code_dummy_KY,state_code_dummy_LA,state_code_dummy_MA,state_code_dummy_MD,state_code_dummy_ME,state_code_dummy_MI,state_code_dummy_MN,state_code_dummy_MO,state_code_dummy_MS,state_code_dummy_NC,state_code_dummy_NE,state_code_dummy_NH,state_code_dummy_NJ,state_code_dummy_NM,state_code_dummy_NV,state_code_dummy_NY,state_code_dummy_OH,state_code_dummy_OK,state_code_dummy_OR,state_code_dummy_PA,state_code_dummy_RI,state_code_dummy_SC,state_code_dummy_TN,state_code_dummy_TX,state_code_dummy_UT,state_code_dummy_VA,state_code_dummy_VT,state_code_dummy_WA,state_code_dummy_WI,state_code_dummy_WV
152,AK,Kenai Peninsula Borough,2019-12-02,220600.0,227209.3,210584.3,204459.3,213584.3,207625.0,227850.0,239100.0,254850.0,449.0,455.0,464.0,472.0,482.0,491.0,437.0,427.0,418.0,0.3,9.1,24.6,36.4,31.8,0.4,88.9,1686,147,20,26,9990.0,6,-3.0,541,6362.0,58708,2977,11431.0,2393,124,1722,23948,3605.0,9995,12702,14977,26341,55400,11.5,11.3,36.8,20.3,5.0,9.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
153,AK,Kenai Peninsula Borough,2019-12-09,227850.0,220600.0,227209.3,210584.3,204459.3,213584.3,239100.0,254850.0,266625.0,437.0,449.0,455.0,464.0,472.0,482.0,427.0,418.0,409.0,0.3,9.1,24.6,36.4,31.8,0.4,88.9,1686,147,20,26,9990.0,6,-3.0,541,6362.0,58708,2977,11431.0,2393,124,1722,23948,3605.0,9995,12702,14977,26341,55400,11.5,11.3,36.8,20.3,5.0,9.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
154,AK,Kenai Peninsula Borough,2019-12-16,239100.0,227850.0,220600.0,227209.3,210584.3,204459.3,254850.0,266625.0,253625.0,427.0,437.0,449.0,455.0,464.0,472.0,418.0,409.0,410.0,0.3,9.1,24.6,36.4,31.8,0.4,88.9,1686,147,20,26,9990.0,6,-3.0,541,6362.0,58708,2977,11431.0,2393,124,1722,23948,3605.0,9995,12702,14977,26341,55400,11.5,11.3,36.8,20.3,5.0,9.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
def convert_frame_to_numpy(df, remove_cols, target_prefix, related_prefix, J,H):
    
    #assemble lag variables.
    y_lag_cols = [f'{target_prefix}_lag_{j}' for j in range(J,0,-1)]
    y_lead_cols = [target_prefix] + [f'{target_prefix}_lead_{h}' for h in range(1,H+1,1)]    
    x_rel_cols = [f'{related_prefix}_lag_{j}' for j in range(J,0,-1)]
    x_rel_lead_cols = [f'{related_prefix}_lag_1'] + [related_prefix] + [f'{related_prefix}_lead_{h}' for h in range(1,H+1,1)] 
    other_cols = [x for x in df.columns if x not in y_lag_cols + y_lead_cols + x_rel_cols + remove_cols + x_rel_lead_cols]
    print(f"Length of other columns = {len(other_cols)}")
    print(other_cols)
    
    
    def get_label_row(row):
        label = np.array([row[remove_cols].values])
        
        return label
    
    def get_xvec_row(row):
        x = np.array([row[y_lag_cols].values])
#         x = np.append(x,[row[x_rel_cols].values],axis=0)

        #Making it nowcast to take future inventory to predict sale price.
        x = np.append(x,[row[x_rel_lead_cols].values],axis=0)
        stat_val = row[other_cols].values
        stat_val = np.tile(stat_val,[J,1])
        stat_val = np.transpose(stat_val)
        x = np.append(x,stat_val,axis=0)
        
        return x
    
    def get_yvec_row(row):
        y = np.array([row[y_lead_cols].values])
        
        return y
            
    X = np.array(df.apply(get_xvec_row, axis = 1))
    y = np.array(df.apply(get_yvec_row, axis = 1))
    label = np.array(df.apply(get_label_row, axis = 1))
    
    return X,y, label
    
remove_cols = ["county_name", "period_begin", "state_code"]
target_prefix = 'median_sale_price'
related_prefix = 'inventory'
J=5
H=3

In [30]:
X_train, y_train, train_label = convert_frame_to_numpy(train_frame, 
                                          remove_cols, 
                                          target_prefix, 
                                          related_prefix, 
                                          J,H)

Length of other columns = 82
['R_INTERNATIONAL_MIG_2019', 'Unemployment_rate_2020', 'PCT_COLL_4_2015_19', 'PCT_COLL_1TO3_2000', 'PCT_HSD_Only_2000', 'R_NET_MIG_2019', 'Med_HH_Income_Percent_of_State_Total_2019', 'GQ_ESTIMATES_2019', 'N_POP_CHG_2019', 'INTERNATIONAL_MIG_2019', 'NET_MIG_2019', 'HSD_Only_2000', 'DOMESTIC_MIG_2019', 'RESIDUAL_2019', 'Deaths_2019', 'COLL_4_2000', 'POP_ESTIMATE_2019', 'LT_HSD_2015_19', 'COLL_1TO3_2000', 'Unemployed_2020', 'NATURAL_INC_2019', 'GQ_ESTIMATES_BASE_2010', 'Employed_2020', 'LT_HSD_2000', 'COLL_4_2015_19', 'HSD_Only_2015_19', 'COLL_1TO3_2015_19', 'Civilian_labor_force_2020', 'CENSUS_2010_POP', 'PCT_LT_HSD_2000', 'R_birth_2019', 'PCT_COLL_1TO3_2015_19', 'PCT_COLL_4_2000', 'Economic_typology_2015', 'R_death_2019', 'state_code_dummy_AK', 'state_code_dummy_AL', 'state_code_dummy_AR', 'state_code_dummy_AZ', 'state_code_dummy_CA', 'state_code_dummy_CO', 'state_code_dummy_CT', 'state_code_dummy_DC', 'state_code_dummy_DE', 'state_code_dummy_FL', 'state_cod

In [31]:
X_test, y_test, test_label = convert_frame_to_numpy(test_frame, 
                                        remove_cols, 
                                        target_prefix, 
                                        related_prefix, 
                                        J,H)

Length of other columns = 82
['R_INTERNATIONAL_MIG_2019', 'Unemployment_rate_2020', 'PCT_COLL_4_2015_19', 'PCT_COLL_1TO3_2000', 'PCT_HSD_Only_2000', 'R_NET_MIG_2019', 'Med_HH_Income_Percent_of_State_Total_2019', 'GQ_ESTIMATES_2019', 'N_POP_CHG_2019', 'INTERNATIONAL_MIG_2019', 'NET_MIG_2019', 'HSD_Only_2000', 'DOMESTIC_MIG_2019', 'RESIDUAL_2019', 'Deaths_2019', 'COLL_4_2000', 'POP_ESTIMATE_2019', 'LT_HSD_2015_19', 'COLL_1TO3_2000', 'Unemployed_2020', 'NATURAL_INC_2019', 'GQ_ESTIMATES_BASE_2010', 'Employed_2020', 'LT_HSD_2000', 'COLL_4_2015_19', 'HSD_Only_2015_19', 'COLL_1TO3_2015_19', 'Civilian_labor_force_2020', 'CENSUS_2010_POP', 'PCT_LT_HSD_2000', 'R_birth_2019', 'PCT_COLL_1TO3_2015_19', 'PCT_COLL_4_2000', 'Economic_typology_2015', 'R_death_2019', 'state_code_dummy_AK', 'state_code_dummy_AL', 'state_code_dummy_AR', 'state_code_dummy_AZ', 'state_code_dummy_CA', 'state_code_dummy_CO', 'state_code_dummy_CT', 'state_code_dummy_DC', 'state_code_dummy_DE', 'state_code_dummy_FL', 'state_cod

In [32]:
X_post_covid, y_post_covid, post_covid_label = convert_frame_to_numpy(post_covid_frame, 
                                        remove_cols, 
                                        target_prefix, 
                                        related_prefix, 
                                        J,H)

Length of other columns = 82
['R_INTERNATIONAL_MIG_2019', 'Unemployment_rate_2020', 'PCT_COLL_4_2015_19', 'PCT_COLL_1TO3_2000', 'PCT_HSD_Only_2000', 'R_NET_MIG_2019', 'Med_HH_Income_Percent_of_State_Total_2019', 'GQ_ESTIMATES_2019', 'N_POP_CHG_2019', 'INTERNATIONAL_MIG_2019', 'NET_MIG_2019', 'HSD_Only_2000', 'DOMESTIC_MIG_2019', 'RESIDUAL_2019', 'Deaths_2019', 'COLL_4_2000', 'POP_ESTIMATE_2019', 'LT_HSD_2015_19', 'COLL_1TO3_2000', 'Unemployed_2020', 'NATURAL_INC_2019', 'GQ_ESTIMATES_BASE_2010', 'Employed_2020', 'LT_HSD_2000', 'COLL_4_2015_19', 'HSD_Only_2015_19', 'COLL_1TO3_2015_19', 'Civilian_labor_force_2020', 'CENSUS_2010_POP', 'PCT_LT_HSD_2000', 'R_birth_2019', 'PCT_COLL_1TO3_2015_19', 'PCT_COLL_4_2000', 'Economic_typology_2015', 'R_death_2019', 'state_code_dummy_AK', 'state_code_dummy_AL', 'state_code_dummy_AR', 'state_code_dummy_AZ', 'state_code_dummy_CA', 'state_code_dummy_CO', 'state_code_dummy_CT', 'state_code_dummy_DC', 'state_code_dummy_DE', 'state_code_dummy_FL', 'state_cod

In [33]:
stack_range = np.array(X_train[0]).shape[0]
print(f"Stack Range : {stack_range}")

Stack Range : 84


In [34]:
X_test

array([array([[207625.0, 213584.3, 204459.3, 210584.3, 227209.3],
       [455.0, 449.0, 437.0, 427.0, 418.0],
       [0.3, 0.3, 0.3, 0.3, 0.3],
       [9.1, 9.1, 9.1, 9.1, 9.1],
       [24.6, 24.6, 24.6, 24.6, 24.6],
       [36.4, 36.4, 36.4, 36.4, 36.4],
       [31.8, 31.8, 31.8, 31.8, 31.8],
       [0.4, 0.4, 0.4, 0.4, 0.4],
       [88.9, 88.9, 88.9, 88.9, 88.9],
       [1686, 1686, 1686, 1686, 1686],
       [147, 147, 147, 147, 147],
       [20, 20, 20, 20, 20],
       [26, 26, 26, 26, 26],
       [9990.0, 9990.0, 9990.0, 9990.0, 9990.0],
       [6, 6, 6, 6, 6],
       [-3.0, -3.0, -3.0, -3.0, -3.0],
       [541, 541, 541, 541, 541],
       [6362.0, 6362.0, 6362.0, 6362.0, 6362.0],
       [58708, 58708, 58708, 58708, 58708],
       [2977, 2977, 2977, 2977, 2977],
       [11431.0, 11431.0, 11431.0, 11431.0, 11431.0],
       [2393, 2393, 2393, 2393, 2393],
       [124, 124, 124, 124, 124],
       [1722, 1722, 1722, 1722, 1722],
       [23948, 23948, 23948, 23948, 23948],
       [3605.

In [35]:
def chunks(a, size):
    arr = iter(a)
    for v in arr:
        tmp = [ v ]
        for i,v in zip( range( size - 1 ), arr ):
            tmp.append( v )
        yield tmp

In [36]:
X_train_stack = list(chunks(np.vstack(X_train), stack_range))
X_test_stack = list(chunks(np.vstack(X_test), stack_range))
label_train_stack = np.expand_dims(np.vstack(train_label),axis=2)
label_test_stack = np.expand_dims(np.vstack(test_label),axis=2)
y_train_stack = np.expand_dims(np.vstack(y_train),axis=2)
y_test_stack = np.expand_dims(np.vstack(y_test),axis=2)

In [37]:
X_train_swap = np.array(X_train_stack).swapaxes(0,1).swapaxes(0,2)
X_test_swap = np.array(X_test_stack).swapaxes(0,1).swapaxes(0,2)
label_train_swap = np.array(label_train_stack).swapaxes(0,1)
label_test_swap = np.array(label_test_stack).swapaxes(0,1)
y_train_swap = np.array(y_train_stack).swapaxes(0,1)
y_test_swap = np.array(y_test_stack).swapaxes(0,1)

In [40]:
X_post_covid_stack = list(chunks(np.vstack(X_post_covid), stack_range))
label_post_covid_stack = np.expand_dims(np.vstack(post_covid_label),axis=2)
y_post_covid_stack = np.expand_dims(np.vstack(y_post_covid),axis=2)

X_post_covid_swap = np.array(X_post_covid_stack).swapaxes(0,1).swapaxes(0,2)
label_post_covid_swap = np.array(label_post_covid_stack).swapaxes(0,1)
y_post_covid_swap = np.array(y_post_covid_stack).swapaxes(0,1)

In [41]:
with open(f"{data_folder}all_model_data_sale_model.npy", 'wb') as f:
    np.save(f, X_train_swap.astype(float))
    np.save(f, y_train_swap.astype(float))
    np.save(f, X_test_swap.astype(float))
    np.save(f, y_test_swap.astype(float))

In [42]:
with open(f"{data_folder}all_model_labels_mapping_sale_model.npy", 'wb') as f:
    np.save(f, label_train_swap)
    np.save(f, label_test_swap)

In [43]:
with open(f"{data_folder}post_covid_inv_data_sale_model.npy", 'wb') as f:
    np.save(f, X_post_covid_swap.astype(float))
    np.save(f, y_post_covid_swap.astype(float))
    
with open(f"{data_folder}post_covid_inv_labels_mapping_sale_model.npy", 'wb') as f:
    np.save(f, label_post_covid_swap)

In [ ]:
label_test_swap[0,:,0].shape